**Importando Librerias**

In [2]:
import pandas as pd

Direcciones de Todos los archivos json referentes a sitios

In [ ]:
list_sitios_path = [
    'datasets/google/sitios/1.json',
    'datasets/google/sitios/2.json',
    'datasets/google/sitios/3.json',
    'datasets/google/sitios/4.json',
    'datasets/google/sitios/5.json',
    'datasets/google/sitios/6.json',
    'datasets/google/sitios/7.json',
    'datasets/google/sitios/8.json',
    'datasets/google/sitios/9.json',
    'datasets/google/sitios/10.json',
    'datasets/google/sitios/11.json'
]

**Juntando Todos los Sitios**

In [ ]:
for i in range(11):
    df = pd.read_json(list_sitios_path[i], lines=True)
    if i == 0:
        sitios = df.copy()
        del df
    else:
        sitios = pd.concat([sitios, df], axis=0, ignore_index=True)
        del df


In [ ]:
sitios = sitios.drop_duplicates(subset=['gmap_id'])

Guardo para tener un respaldo de la Data Toda Junta sin Modificar

In [ ]:
sitios.to_parquet('datasets/google/sitios/sitios_google_todas_columnas_originales.parquet')

Creando y Desempaquetando la lista de relative_results

In [ ]:
relative_results = sitios[['gmap_id', 'relative_results']]

In [ ]:
relative_results = relative_results.explode('relative_results')

Guardando la tabla `google_relative_results_sitios.parquet`

In [ ]:
relative_results.to_parquet('datasets/google/sitios/relative_results.parquet')

del relative_results

Creando y Guardando a `misc_sitios` pero todavia anidada

In [ ]:
misc_sitios = sitios[['gmap_id', 'MISC']]

Donde MISC sea NULO no hay Información Se pueden Borrar ya que esta tabla es SOLO MISCELANEOS

In [19]:
misc_sitios = misc_sitios.dropna(subset=['MISC'])

Defino una Funcion que convierte los Diccionarios en Listas para poder Aplicar `explode` despues.  Aca sólo se van a tomar aquellos pares `claves-value` del diccionario donde `value is not None`

In [10]:
def dict_to_list(diccionario:dict) -> list:
    """dict_to_list Devuelve el Diccionario de la columna `MISC` y lo devuelve
                    como una lista de String "key:Value" Siempre y cuando el 
                    value de la pareja clave-valor no sea Nulo

    Args:
        diccionario (dict): diccionario recibido

    Returns:
        list: Lista a devolver
    """
    return [f"{key}: {value}" for key, value in diccionario.items() if value is not None]

In [22]:
misc_sitios['lista_misc'] = misc_sitios['MISC'].apply(dict_to_list)

misc_sitios.head()

,gmap_id,MISC,lista_misc
0,0x88f16e41928ff687:0x883dad4fd048e8f8,{'Accessibility': ['Wheelchair accessible entr...,[Accessibility: ['Wheelchair accessible entran...
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,{'Accessibility': ['Wheelchair accessible entr...,[Accessibility: ['Wheelchair accessible entran...
3,0x80c2c89923b27a41:0x32041559418d447,"{'Accessibility': None, 'Activities': None, 'A...",[Payments: ['Checks' 'Debit cards' 'Credit car...
4,0x80c2c632f933b073:0xc31785961fe826a6,"{'Accessibility': None, 'Activities': None, 'A...",[Service options: ['In-store pickup']]
5,0x80c2cf163db6bc89:0x219484e2edbcfa41,{'Accessibility': ['Wheelchair accessible entr...,[Accessibility: ['Wheelchair accessible entran...


Debo Borrar la columna MISC, aplicar explode al dataframe con respecto a la columna `lista_misc` y luego renombrar columna y ya estaría lista la tabla

In [23]:
misc_sitios = misc_sitios.drop(columns=['MISC'])

misc_sitios = misc_sitios.explode('lista_misc')

misc_sitios.rename(columns={'lista_misc': 'misc'}, inplace=True)
misc_sitios.head()

,gmap_id,misc
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Accessibility: ['Wheelchair accessible entrance']
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Health & safety: ['Mask required' 'Staff requi...
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Planning: ['Quick visit']
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Service options: ['In-store shopping' 'Same-da...
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Accessibility: ['Wheelchair accessible entrance']


Hay que guardar la Tabla `google_misc_sitios`

In [25]:
misc_sitios.to_parquet('datasets/google/sitios/google_misc_sitios.parquet')

In [ ]:
misc_sitios.to_parquet('datasets/google/sitios/misc_sitios_anidada.parquet')

In [ ]:
del misc_sitios

Creando y Guardando a cotegories_sitios pero todavia con nombres de categorias, ya que el Maestro de Categorias no se ha creado aún

In [ ]:
categories_sitios = sitios[['gmap_id', 'category']].explode('category')

In [ ]:
categories_sitios.to_parquet('datasets/google/sitios/categories_sitios_google_todavia_nombres.parquet')

In [ ]:
del categories_sitios

Borrando las columnas `MISC`, `relative_results` y `category`

In [ ]:
sitios = sitios.drop(columns=['MISC', 'relative_results', 'category'])

**Funcion para descomponer la Lista de los Horarios**

In [ ]:
def retornar_horario(campo:list[str], dia:str) -> str:
    """retornar_horario: funcion de apoyo a los campos que representan el horario de
                         cada dia: monday, tuesday....sunday

    Args:
        campo (list[list]): lista de la columna `hours` 
        dia (str): dia de la semana a consultar en campo

    Returns:
        str: Horario del dia especifico que se consulta
    """
    try:
        for i in range(7):
            if campo[i][0].lower() != dia.lower():
                continue
            buscado = campo[i][1]
            break
        return buscado
    except:
        return 'No Disponible'

**Creando las Columnas de los Horarios**

In [ ]:
sitios['monday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Monday'))

sitios['tuesday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Tuesday'))

sitios['wednesday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Wednesday'))

sitios['thursday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Thursday'))

sitios['friday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Friday'))

sitios['saturday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Saturday'))

sitios['sunday'] = sitios['hours'].apply(lambda x: retornar_horario(x, dia='Sunday'))

sitios.head()

,name,address,gmap_id,description,latitude,longitude,avg_rating,num_of_reviews,price,hours,state,url,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",Open ⋅ Closes 6PM,https://www.google.com/maps/place//data=!4m2!3...,8AM–6PM,8AM–6PM,8AM–12PM,8AM–6PM,8AM–6PM,8AM–12PM,Closed
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,4.5,6,None,None,Open now,https://www.google.com/maps/place//data=!4m2!3...,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",Open ⋅ Closes 6PM,https://www.google.com/maps/place//data=!4m2!3...,Closed,6:30AM–6PM,6:30AM–6PM,6:30AM–6PM,6:30AM–6PM,6:30AM–6PM,7AM–12PM
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",Open ⋅ Closes 5PM,https://www.google.com/maps/place//data=!4m2!3...,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,Closed,Closed
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",Open ⋅ Closes 5PM,https://www.google.com/maps/place//data=!4m2!3...,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,Closed,Closed


Borrando la Columna `hours`

In [ ]:
sitios.drop('hours', axis=1, inplace=True)

In [ ]:
sitios.head(1)

,name,address,gmap_id,description,latitude,longitude,avg_rating,num_of_reviews,price,state,url,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.3883,-83.3571,4.9,16,None,Open ⋅ Closes 6PM,https://www.google.com/maps/place//data=!4m2!3...,8AM–6PM,8AM–6PM,8AM–12PM,8AM–6PM,8AM–6PM,8AM–12PM,Closed


Guardando la tabla de `sitios`

In [ ]:
sitios.to_parquet('datasets/google/sitios/google_sitios.parquet')